# 05 Nondeterminisitc 비결정적

DFA와 NFA를 비교하여 생각해 보기로 하자.

DFA와 NFA모두 $(Q,\Sigma,q_0, \delta, F)$의 다섯순서쌍(5-tuple)로 표현된다는 점에서는 같지만
DFA와 NFA의 차이는 전이함수인 $\delta$의 형식이 다르다는 점이다.

 * $\delta : Q\times \Sigma \to \Sigma$ $\qquad\qquad\qquad$ DFA의 전이함수
 * $\delta : Q\times (\Sigma\cup\{\varepsilon\}) \to 2^Q$ $\qquad$ $\varepsilon$전이를 허용하는 NFA의 전이함수
 * $\delta : Q\times \Sigma \to 2^Q$ $\qquad~\quad\qquad$ $\varepsilon$전이가 없는 NFA의 전이함수

책에는 $\varepsilon$전이를 허용하는 NFA를 중심으로 설명하지만 $\varepsilon$이 없는 경우에 설명하기가 간단한 부분이 많기 때문에
책과는 달리 $\varepsilon$전이가 없는 NFA를 중심으로 설명을 진행하겠다.

Haskell 코드에서는 $Q=\{q_0,q_1,\ldots\}$ 대신 그냥 정수 타입은 `Int`를 쓰고 $\Sigma$대신 컴퓨터 프로그래밍 언어에서 제공하는 문자 타입인 `Char`를 쓰기로 정해 놓고 진행하므로
다섯순서쌍 중 앞으 둘을 빼고 나머지 세 항목의 정보, 즉 시작 상태, 전이함수, 그리고 종료상태만 나타내면 된다.
그리고 Haskell 코드에서는 시작상태를 전이함수에서 사용되는 상태들 중에 최소값으로 하기로 규칙을 정하도록 하자.
그러니까 상태를 0,1,2,3,4 이렇게 다섯개를 쓴다면 시작 상태는 0으로 하고 1,2,3 이렇게 세 개를 쓴다면 시작 상태는 1로 하기로 한다는 말이다.

In [4]:
type Q     = Int
type Sigma = Char
type NFA   = (Int, Delta, [Int])
type Delta = [(Q,Sigma,Q)]

DFA의 타입을 굳이 따로 정의하지 않는 이유는 NFA가 DFA를 정의를 포함하는 것으로 볼 수 있기 때문이다.

즉, NFA의 전이함수가 모든 $q\in Q$, $c\in\Sigma$에 대해 $|\delta(q,c)|\le1$를 만족하면 DFA인 것이다.

In [5]:
import Data.List

isDFA :: NFA -> Bool
isDFA (_,d,_)= all ((<= 1) . length) (groupBy (\(q1,c1,_) (q2,c2,_) -> (q1,c1)==(q2,c2)) d)

In [11]:
d3 = [(0,'a',1),(0,'b',2),(2,'a',1)] -- DFA의 성질을 만족하는 전이함수
d4 = [(0,'a',1),(0,'a',2),(1,'b',1)] -- DFA의 성질을 만족하는 전이함수
nfa3 = (0, d3, [1])
nfa4 = (0, d4, [1])

mapM_ print $ groupBy (\(q1,c1,_) (q2,c2,_) -> (q1,c1)==(q2,c2)) d3
isDFA nfa3

mapM_ print $ groupBy (\(q1,c1,_) (q2,c2,_) -> (q1,c1)==(q2,c2)) d4
isDFA nfa4

[(0,'a',1)]
[(0,'b',2)]
[(2,'a',1)]

True

[(0,'a',1),(0,'a',2)]
[(1,'b',1)]

False

그러면 어떤 $x$라는 문자열(String)이 주어졌을 때 NFA가 이 언어를 받아들이는지 받아들이지 않는지 판별하는 프로그램을 작성해 보자.
지난 수업시간에 질문이 나와서 설명했던 $\hat\delta$ 함수를 Haskell 프로그램으로 작성하기만 하면 거의 다 된 것이다.
왜냐하면 $\hat{\delta}(q_0,x)$의 결과가 $F$와 교집합을 갖는지만 검사하면 되기 때문이다.

일단 주어진 NFA로부터 순서쌍 리스트의 형태로 저장된 전이함수를 실제 함수 $\delta$로 변환하는 하스켈 프로그램을 작성하고
그것을 바탕으로 $\hat\delta$ 함수를 Haskell로 작성해 보자.

In [12]:
delta :: NFA -> (Q,Sigma) -> [Q]
delta (_,d,_) (q,c)= [p1 | (q1,c1,p1)<-d, (q,c)==(q1,c1)]

hat :: ( (Q,Sigma) -> [Q] ) -> (Q,String) -> [Q]
hat f (q, "" ) = [q]
hat f (q, a:y) = [p | q1 <- f(q,a), p <- hat f (q1,y) ]

$\displaystyle\big\{xaa \mid x\in\{a,b\}^{*}\big\}$

In [31]:
nfa1 = (1, d1, [3])
d1 = [ (1,'a',1), (1,'b',1), (1,'a',2), (2,'a',3) ]

δ1 = delta nfa1
:type δ1

δ1' = hat δ1
:type δ1'

δ1 :: (Q, Sigma) -> [Q]

δ1' :: (Q, String) -> [Q]

In [32]:
δ1(1,'a')

δ1'(1,"a")
δ1'(1,"aa")
δ1'(1,"ab")
δ1'(1,"aba")
δ1'(1,"abaa")

[1,2]

[1,2]

[1,2,3]

[1]

[1,2]

[1,2,3]

In [36]:
import Data.List

accepts (q0, d, fs) x = (δ'(q0,x) `intersect` fs) /= []
  where
    δ  = delta (q0, d, fs)
    δ' = hat δ

In [37]:
nfa1 `accepts` "a"
nfa1 `accepts` "aa"
nfa1 `accepts` "ab"
nfa1 `accepts` "aba"
nfa1 `accepts` "abaa"

False

True

False

False

True

$\displaystyle\big\{xaaybbz \mid x,y,z\in\{a,b\}^{*} \big\}$